In [8]:
from owlready2 import get_ontology
import os
main_dir = os.getcwd().split("BERTMap")[0] + "BERTMap"
import sys
sys.path.append(main_dir)
import json
import re
from bertmap.utils import uniqify
from copy import deepcopy
import random

In [2]:
snomed = get_ontology(main_dir + "/data/largebio/ontos/snomed2fma.small.owl").load()
snomed_ori = get_ontology(main_dir + "/data/largebio/ontos/SNOMED.ori.owl").load()

In [5]:
# remove (branch) for every label in Original SNOMED
pa = r"^(.+) \(.+?\)$"  # label (branch) -> label
for cl in snomed_ori.classes():
    labs = []
    for x in cl.label:
        labs.append(re.findall(pa, x)[0])
    if labs:
        cl.label = labs
    else:
        print(cl, cl.label)
        break

In [26]:
# add labels to snomed to form snomed+
empty = []
unique = 0
multiple = 0
added = 0
for cl in snomed.classes():
    labs = deepcopy(list(cl.label))
    classes = []
    for x in cl.label:
        pref_cls = list(snomed_ori.search(prefLabel=x, _case_sensitive=False))
        alt_cls = list(snomed_ori.search(altLabel=x, _case_sensitive=False))
        lab_cls = list(snomed_ori.search(label=x, _case_sensitive=False))
        classes += pref_cls + alt_cls + lab_cls
        assert len(classes) >= len(pref_cls) + len(alt_cls) + len(lab_cls)
    classes = uniqify(classes)
    # remove classes from snomed instead of original SNOMED
    classes = [c for c in classes if "http://www.ihtsdo.org/snomed#" not in c.iri]

    if len(classes) == 0: empty.append(cl)
    elif len(classes) == 1: unique += 1
    elif len(classes) > 1: multiple += 1

    for c in classes:
        labs += list(c.label) + list(c.prefLabel) + list(c.altLabel)

    labs = uniqify(labs)
    if len(labs) > 1: 
        added += 1
        cl.label = labs

        

        


KeyboardInterrupt: 

In [ ]:
classes_plus = list(snomed.classes())
